# Enable APIs (Container, Kubernetes) in Google Cloud Platform

# Create cluster in console

# Install Google Cloud SDK

# Configure Docker to use gcloud tool to authenticate requests
> gcloud auth configure-docker

# Download auth token key from console and put it in the directory
> API & Services -> Credentials: create credentials > Service account key

# Fetch cluster endpoint and auth data and update kubeconfig entry

> gcloud container clusters get-credentials mm-docker-models-gke --zone us-east4-a [--project <project name>]

# Configure firewall to allow the node port range (30000-32767)
> gcloud compute firewall-rules create gke-rule --allow tcp:30000-32767


# Configure 'GCP' section in cli.properties


In [1]:
from mm_docker_lib import *
initConfig("GCP")

Loading configuration properties...
key file: sas-tyfrec-209092a0e3d2.json
project name: sas-tyfrec
Login into GCP GCR...
Login GCP GCR succeeded!
Initializing kubernetes configuration... gke_sas-tyfrec_us-central1-a_eyap-mas
Initialized kubernetes configuration.
  verbose: True
  model.repo.host: http://gtp-viya.openstack.sas.com
  provider.type: GCP
  base.repo: gcr.io/sas-tyfrec/
  base.repo.web.url: console.cloud.google.com/gcr/images/sas-tyfrec
  kubernetes.context: gke_sas-tyfrec_us-central1-a_eyap-mas


True

In [3]:
listmodel("svm")

Getting model list...
Model name svm (pipeline 1)
Model UUID 4ac36c21-afbd-40a8-820b-0c4cf6b41430
Model version 1.1
Project name EYAP_tyfrec_test
Score Code Type ds2MultiType
Image URL (not verified): gcr.io/sas-tyfrec/svm-pipeline-1_4ac36c21-afbd-40a8-820b-0c4cf6b41430:latest
Guides: > python mm_docker_cli.py publish <uuid>
Guides: > python mm_docker_cli.py launch <image url>
Guides: > python mm_docker_cli.py score <image url> <input file>


In [5]:
# returns the docker image URL of the container we created and pushed to our GCP repo
container = publish("4ac36c21-afbd-40a8-820b-0c4cf6b41430")

Images folder: /net/ge.unx.sas.com/vol/vol610/u61/tyfrec/eyapmas/images
Model zip file has been downloaded at /net/ge.unx.sas.com/vol/vol610/u61/tyfrec/eyapmas/images/model-4ac36c21-afbd-40a8-820b-0c4cf6b41430/model-4ac36c21-afbd-40a8-820b-0c4cf6b41430.zip
SVM (Pipeline 1) 1.1 ds2MultiType
The name has been normalized to svm-pipeline-1
Template folder: /net/ge.unx.sas.com/vol/vol610/u61/tyfrec/eyapmas/template
Copying astore from shared directory...
Make sure that the shared directory has been mounted locally
For Windows, the local directory will be at z drive
For Linux, the local directory will be /astore
/astore/_1Q90JQSWYWIDH1M94NNHHBBWET.astore
/net/ge.unx.sas.com/vol/vol610/u61/tyfrec/eyapmas/images/model-4ac36c21-afbd-40a8-820b-0c4cf6b41430
Building image...
svm-pipeline-1_4ac36c21-afbd-40a8-820b-0c4cf6b41430:1.1
svm-pipeline-1_4ac36c21-afbd-40a8-820b-0c4cf6b41430:latest
Tag the image into a repository sha256:f10bd43b71
Docker repository URL:  gcr.io/sas-tyfrec/
gcr.io/sas-tyfrec

In [6]:
# launches the container into GCP Kubernetes engine.
# Returns a tuple of the form: (kubernetes-deployment-id, http://URL-of-service:1234)
deployment = launch(container)

Validating image repository url...
Completed validation.
Launching container instance...
gcr.io/sas-tyfrec/svm-pipeline-1_4ac36c21-afbd-40a8-820b-0c4cf6b41430:latest
svm-pipeline-1
Deployment created. 
Deployment name:  svm-pipeline-1-pd45cs
Service created.
Getting service url...
Service URL:  http://35.188.175.53:32207
Checking whether the instance is up or not...
1 ==Sleep 10 seconds...
Checking whether the instance is up or not...
Instance is up!
Guides: > python mm_docker_cli.py execute  http://35.188.175.53:32207 <input file>
Guides: > python mm_docker_cli.py stop svm-pipeline-1-pd45cs


In [8]:
# Scores the input file against the scoring service.
# Returns an id of the score
test_id = execute(deployment[1], "hmeq_test.csv")

Performing scoring in the container instance...
service_url: http://35.188.175.53:32207
csv_file: hmeq_test.csv
{'id': '1553270697.7237422', 'status': 201}
The test_id from score execution: 1553270697.7237422
Guides: > python mm_docker_cli.py query http://35.188.175.53:32207/ 1553270697.7237422


In [9]:
query(service_url=deployment[1],test_id=test_id)

service_url: http://35.188.175.53:32207
test_id: 1553270697.7237422
The test result has been retrieved and written into file 1553270697.7237422.csv
Head is the first 5 lines
EM_CLASSIFICATION,EM_EVENTPROBABILITY,EM_PROBABILITY,I_BAD,P_BAD0,P_BAD1,_P_,_WARN_
0                               ,0.00019667529,0.9998033,0                               ,0.9998033,0.00019667529,1.0000238,
0                               ,0.00021125952,0.99978876,0                               ,0.99978876,0.00021125952,0.9999947,
0                               ,0.0001889327,0.99981105,0                               ,0.99981105,0.0001889327,1.0000393,
1                               ,0.7044128,0.7044128,1                               ,0.29558718,0.7044128,-1.225961,
Guides: 1) remember to stop instance after uage. You can find the deployment name by running
    > kubectl get deployment
Then execute: > python mm_docker_cli.py stop <deployment_name>
Guides: 2) if result file includes error message, you could fi

'1553270697.7237422.csv'

In [10]:
stop(deployment_name=deployment[0])

svm-pipeline-1-pd45cs
Deleting service svm-pipeline-1-pd45cs
deleted svc/svm-pipeline-1-pd45cs from ns/default
Deleting app deployment... svm-pipeline-1-pd45cs
Deletion succeeded


In [11]:
score(container, "hmeq_test.csv")

Validating image repository url...
Completed validation.
Launching container instance...
gcr.io/sas-tyfrec/svm-pipeline-1_4ac36c21-afbd-40a8-820b-0c4cf6b41430:latest
svm-pipeline-1
Deployment created. 
Deployment name:  svm-pipeline-1-180e5g
Service created.
Getting service url...
Service URL:  http://35.188.175.53:32257
Checking whether the instance is up or not...
1 ==Sleep 10 seconds...
Checking whether the instance is up or not...
Instance is up!
Guides: > python mm_docker_cli.py execute  http://35.188.175.53:32257 <input file>
Guides: > python mm_docker_cli.py stop svm-pipeline-1-180e5g
Performing scoring in the container instance...
service_url: http://35.188.175.53:32257
csv_file: hmeq_test.csv
{'id': '1553270798.3480272', 'status': 201}
The test_id from score execution: 1553270798.3480272
Guides: > python mm_docker_cli.py query http://35.188.175.53:32257/ 1553270798.3480272
service_url: http://35.188.175.53:32257
test_id: 1553270798.3480272
The test result has been retrieved an